In [1]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('research_traffic_multiple_labels.csv')
df.shape

(60000, 12)

In [2]:
print(df.dtypes)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()

Count of Source IP        int64
Port Count                int64
Pair Count Ratio        float64
Packet Count Diff         int64
Lookup Count Diff         int64
Protocol                  int64
Average Packet Count    float64
Average Byte Count      float64
Packet Std Dev          float64
Byte Std Dev            float64
Duration per Flow       float64
Label                     int64
dtype: object
10000
10000


,Count of Source IP,Port Count,Pair Count Ratio,Packet Count Diff,Lookup Count Diff,Protocol,Average Packet Count,Average Byte Count,Packet Std Dev,Byte Std Dev,Duration per Flow,Label
0,545,545,0.0,2,43,6,0.027523,4.788991,0.163601,28.466647,0.668020,3
1,545,545,0.0,10,0,6,0.009174,1.596330,0.095342,16.589551,0.419081,3
2,52,52,0.0,3,0,6,0.000000,0.000000,0.000000,0.000000,0.018792,3
3,545,545,0.0,13,21,6,0.023853,4.150459,0.355016,61.772825,2.946281,3
4,545,545,0.0,2,131,6,0.027523,4.788991,0.163601,28.466647,2.674461,3


In [3]:
# obj_cols=df_XGB.dtypes[df_XGB.dtypes == "object"].index.values.tolist()
# print(obj_cols)

# from sklearn.preprocessing import LabelEncoder
# #Encode labels of multiple columns at once

# df_XGB[obj_cols] = df_XGB[obj_cols].astype(str)
# df_XGB[obj_cols] = df_XGB[obj_cols].apply(LabelEncoder().fit_transform)
# #
# # Print head
# #
# print(df_XGB.dtypes)
# df_XGB.head()

In [4]:
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score, make_scorer
from sklearn.metrics import f1_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, cross_validate

y = df.Label
X = df.drop(['Label'],axis=1)
num_classes = len(np.unique(y))

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=num_classes, eval_metric='mlogloss', use_label_encoder=False)

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

Dataset normalized using MinMaxScaler.


In [5]:
# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []
start = datetime.now()

# Perform cross-validation manually
for train_index, test_index in kf.split(X_train_cv):
    X_train, X_test = X_train_cv.iloc[train_index], X_train_cv.iloc[test_index]
    y_train, y_test = y_train_cv.iloc[train_index], y_train_cv.iloc[test_index]

    # Fit the model
    xgb_model.fit(X_train, y_train)

    # Predict on training and test data
    y_train_pred = xgb_model.predict(X_train)
    y_test_pred = xgb_model.predict(X_test)

    # Compute confusion matrix for the test data
    cm = confusion_matrix(y_test, y_test_pred)
    confusion_matrices.append(cm)

    # Compute and store scores
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))
    precisions.append(precision_score(y_test, y_test_pred, average='weighted'))
    recalls.append(recall_score(y_test, y_test_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_test_pred, average='weighted'))
    confusion_matrices.append(confusion_matrix(y_test, y_test_pred))

# Test the final model on unseen data
y_unseen_pred = xgb_model.predict(X_unseen_test)
unseen_accuracy = accuracy_score(y_unseen_test, y_unseen_pred)
unseen_precision = precision_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_recall = recall_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_f1 = f1_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_conf_matrix = confusion_matrix(y_unseen_test, y_unseen_pred)
end = datetime.now()

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:47:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:47:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:47:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "

In [6]:
# Print the metrics for each fold
for i in range(10):
    print(f"Fold {i+1}:")
    print(f"Training Accuracy: {train_accuracies[i]}")
    print(f"Testing Accuracy: {test_accuracies[i]}")
    print(f"Precision: {precisions[i]}")
    print(f"Recall: {recalls[i]}")
    print(f"F1-Score: {f1_scores[i]}")
    print(f"Confusion Matrix:\n{confusion_matrices[i]}\n")

print(f"Average Training Accuracy: {sum(train_accuracies) / 10}")
print(f"Average Testing Accuracy: {sum(test_accuracies) / 10}")
print(f"Average Precision: {sum(precisions) / 10}")
print(f"Average recall: {sum(recalls) / 10}")
print(f"Average F1 Score: {sum(f1_scores) / 10}")

# Print the performance on the unseen data
print("Performance on Unseen Data:")
print(f"Accuracy: {unseen_accuracy}")
print(f"Precision: {unseen_precision}")
print(f"Recall: {unseen_recall}")
print(f"F1-Score: {unseen_f1}")
print(f"Confusion Matrix:\n{unseen_conf_matrix}")
print("excution time: ", end-start)

Fold 1:
Training Accuracy: 0.9683564814814815
Testing Accuracy: 0.9539583333333334
Precision: 0.9557467264723057
Recall: 0.9539583333333334
F1-Score: 0.952409076659956
Confusion Matrix:
[[577  40   1  28   0 115]
 [  6 809   0   0   0   5]
 [  2   0 771   0   0   0]
 [  0   0   0 794   0   0]
 [  3   0   0   0 804   0]
 [ 15   5   0   1   0 824]]

Fold 2:
Training Accuracy: 0.968912037037037
Testing Accuracy: 0.95125
Precision: 0.9536749084814267
Recall: 0.95125
F1-Score: 0.9499505817035799
Confusion Matrix:
[[577  40   1  28   0 115]
 [  6 809   0   0   0   5]
 [  2   0 771   0   0   0]
 [  0   0   0 794   0   0]
 [  3   0   0   0 804   0]
 [ 15   5   0   1   0 824]]

Fold 3:
Training Accuracy: 0.968912037037037
Testing Accuracy: 0.9558333333333333
Precision: 0.9577613900516057
Recall: 0.9558333333333333
F1-Score: 0.9547679454984541
Confusion Matrix:
[[640  31   0  42   2 114]
 [  9 754   1   0   0  13]
 [  1   0 811   0   0   0]
 [  1   0   0 813   0   0]
 [  1   0   0   0 784   0]
 

In [8]:
# import pickle
# with open('xgb_multi.pkl', 'wb') as file:
#     pickle.dump(xgb_model, file)